In [1]:
import requests
import helpers

def sparqlQuery(query):
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    return requests.get(url, params={'query': query, 'format': 'json'}).json()

def toDict(results):
    var = results['head']['vars']
    if (len(var) > 2):
        raise ValueError('More than 2 results vars')
    dic = {}
    for item in results['results']['bindings']:
        dic.update({ item[var[0]]['value'] : item[var[1]]['value'] })

    return dic




In [2]:
def retriveDepicts(painting=None):
    if painting == None:
        return None
    
    data = sparqlQuery("SELECT ?depicts ?depictsLabel WHERE { " \
            "    wd:" + painting + " wdt:P180 ?depicts." \
            "    SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }" \
            "}" \
            "LIMIT 100")

    return dict(map(lambda kv: (kv[0][kv[0].rindex("/")+1:], kv[1]), toDict(data).items()))

In [4]:
retriveDepicts(painting="Q2270955")

{'Q8441': 'man',
 'Q23640': 'head',
 'Q41370': 'David',
 'Q48422': 'cadaver',
 'Q192785': 'Goliath'}

In [ ]:
def retrieveSuperclass(obj=None):
    if obj == None:
        return None
    data = sparqlQuery("SELECT ?instance ?instanceLabel WHERE {" \
            "   wd:" + obj + " wdt:P279|wdt:P31 ?instance." \
            "   SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }" \
            "}" \
            "LIMIT 100")
    return dict(map(lambda kv: (kv[0][kv[0].rindex("/")+1:], kv[1]), toDict(data).items()))


In [8]:
retrieveSuperclass('Q8441')

ValueError: too many values to unpack (expected 2)

In [6]:
sparqlQuery("SELECT ?instance ?instanceLabel WHERE {" \
            "wd:Q23640 wdt:P279|wdt:P31 ?instance." \
            "SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }" \
            "}" \
            "LIMIT 100")

{'head': {'vars': ['instance', 'instanceLabel']},
 'results': {'bindings': [{'instance': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q188193'},
    'instanceLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'sistema endocri'}},
   {'instance': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q17781690'},
    'instanceLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'cardinal body part'}}]}}